In [0]:
%run "./00_config_storage"

In [0]:
from pyspark.sql.functions import when, col, regexp_replace, trim

df = spark.read.csv("abfss://raw@vanbavietdatalake.dfs.core.windows.net/city.csv", header=True, inferSchema=True)
#rename column
df_rename_col=df.withColumnRenamed("lat","latitude").withColumnRenamed("lon","longitude")
#remove null value
df_fix_null = df_rename_col.withColumn("state_clean",when(col("state").isNull(),trim(
            regexp_replace(col("city"), r"(?i)^thành\s+ph[ốo]\s*", "")
        )).otherwise(col("state")))

df_fix_null.select("country","city","state_clean","latitude","longitude").show(truncate=False)
df_fix_null.write.mode("overwrite").parquet("abfss://raw@vanbavietdatalake.dfs.core.windows.net/city_clean.parquet")
